In [1]:
from dotenv import load_dotenv

load_dotenv()

import os
import datasets
from tqdm import tqdm
from langchain.docstore.document import Document as LangchainDocument
from langchain_openai import ChatOpenAI

# Import shared functions
from rag_helpers import (
    read_jsonl,
    load_embeddings,
    answer_with_rag,
    run_rag_tests,
    RAG_PROMPT_TEMPLATE,
    EVALUATION_PROMPT
)


load_dir = "datasets_local/20250914_145157"

# Load initial corpus
ds = read_jsonl(os.path.join(load_dir, "initial_corpus.jsonl"))
RAW_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]})
    for doc in tqdm(ds)
]

# Load eval dataset - CHOOSE ONE:

# Option 1: Use the HuggingFace dataset with 65 questions (RECOMMENDED)
eval_path_hf = os.path.join(load_dir, "eval_dataset_hf.jsonl")
if os.path.exists(eval_path_hf):
    eval_dataset = datasets.Dataset.from_list(read_jsonl(eval_path_hf))
    print(f"Loaded {len(eval_dataset)} eval questions from HuggingFace dataset")
else:
    # Download and save for next time
    print("Downloading HuggingFace eval dataset...")
    eval_dataset = datasets.load_dataset("m-ric/huggingface_doc_qa_eval", split="train")
    # Save it for future use
    write_jsonl(eval_path_hf, [item for item in eval_dataset])

# Option 2: Use your own generated dataset with only 6 questions (NOT RECOMMENDED for evaluation)
# eval_path = os.path.join(load_dir, "qa_filtered.jsonl")
# eval_dataset = datasets.Dataset.from_list(read_jsonl(eval_path))
# print(f"Loaded {len(eval_dataset)} eval questions from generated dataset")

# # Setup LLM
# READER_MODEL_NAME = "gpt-4o-mini"
# READER_LLM = ChatOpenAI(model=READER_MODEL_NAME, temperature=0.1, max_tokens=512)

100%|██████████| 2647/2647 [00:00<00:00, 434720.34it/s]

Loaded 65 eval questions from HuggingFace dataset


# 3. Benchmarking the RAG system

The RAG system and the evaluation datasets are now ready. The last step is to judge the RAG system's output on this evaluation dataset.

To this end, __we setup a judge agent__. ⚖️🤖

Out of [the different RAG evaluation metrics](https://docs.ragas.io/en/latest/concepts/metrics/index.html), we choose to focus only on Answer Correctness since it is the best end-to-end metric of our system's performance.

> We use GPT4 as a judge for its empirically good performance, but you could try with other models such as [kaist-ai/prometheus-13b-v1.0](https://huggingface.co/kaist-ai/prometheus-13b-v1.0) or [BAAI/JudgeLM-33B-v1.0](https://huggingface.co/BAAI/JudgeLM-33B-v1.0).

💡 _In the evaluation prompt, we give a detailed description each metric on the scale 1-5, as is done in [Prometheus's prompt template](https://huggingface.co/kaist-ai/prometheus-13b-v1.0): this helps the model ground its metric precisely. If instead you give the judge LLM a vague scale to work with, the outputs will not be consistent enough between different examples._

💡 _Again, prompting the LLM to output rationale before giving its final score gives it more tokens to help it formalize and elaborate a judgement._

In [2]:


from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import SystemMessage


evaluation_prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a fair evaluator language model."),
        HumanMessagePromptTemplate.from_template(EVALUATION_PROMPT),
    ]
)

In [3]:
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
import os
import json
from tqdm import tqdm

eval_chat_model = ChatOpenAI(model="gpt-4o", temperature=0)
evaluator_name = "GPT4"


def evaluate_answers(
    answer_path: str,
    eval_chat_model,
    evaluator_name: str,
    evaluation_prompt_template: ChatPromptTemplate,
) -> None:
    """Evaluates generated answers. Modifies the given answer file in place for better checkpointing."""
    answers = []
    if os.path.isfile(answer_path):  # load previous generations if they exist
        answers = json.load(open(answer_path, "r"))

    for experiment in tqdm(answers):
        if f"eval_score_{evaluator_name}" in experiment:
            continue

        eval_prompt = evaluation_prompt_template.format_messages(
            instruction=experiment["question"],
            response=experiment["generated_answer"],
            reference_answer=experiment["true_answer"],
        )
        eval_result = eval_chat_model.invoke(eval_prompt)
        feedback, score = [
            item.strip() for item in eval_result.content.split("[RESULT]")
        ]
        experiment[f"eval_score_{evaluator_name}"] = score
        experiment[f"eval_feedback_{evaluator_name}"] = feedback

        with open(answer_path, "w") as f:
            json.dump(answers, f)

🚀 Let's run the tests and evaluate answers!👇

In [4]:
from ragatouille import RAGPretrainedModel
from langchain_openai import ChatOpenAI
import os

# Create output directory if it doesn't exist
if not os.path.exists("output"):
    os.mkdir("output")

# Define reader models
READER_MODELS = {
    "gpt-4o-mini": ChatOpenAI(model="gpt-4o-mini", temperature=0.1, max_tokens=512),
    "gpt-4o": ChatOpenAI(model="gpt-4o", temperature=0.1, max_tokens=512),
}

# Minimal configuration - focus on what matters most
CONFIGS = [
    {
        "chunk": 200,
        "reader": "gpt-4o-mini",
        "rerank": False,
        "embeddings": "text-embedding-3-small",
    },
    
    {
        "chunk": 200,
        "reader": "gpt-4o-mini",
        "rerank": True,
        "embeddings": "text-embedding-3-small",
    },
    # {
    #     "chunk": 400,
    #     "reader": "gpt-4o-mini",
    #     "rerank": False,
    #     "embeddings": "text-embedding-3-small",
    # },

    # {
    #     "chunk": 400,
    #     "reader": "gpt-4o-mini",
    #     "rerank": True,
    #     "embeddings": "text-embedding-3-small",
    # },

    # {
    #     "chunk": 200,
    #     "reader": "gpt-4o",
    #     "rerank": False,
    #     "embeddings": "text-embedding-3-small",
    # },
    # {
    #     "chunk": 200,
    #     "reader": "gpt-4o",
    #     "rerank": True,
    #     "embeddings": "text-embedding-3-small",
    # },

    # {
    #     "chunk": 400,
    #     "reader": "gpt-4o",
    #     "rerank": False,
    #     "embeddings": "text-embedding-3-small",
    # },
    # {
    #     "chunk": 400,
    #     "reader": "gpt-4o",
    #     "rerank": True,
    #     "embeddings": "text-embedding-3-small",
    # },
    # Add these if you want to test reranking with best configs
    # {"chunk": 200, "reader": "gpt-4o-mini", "rerank": True, "embeddings": "text-embedding-3-small"},
]

/var/folders/pd/9hqpxwh512sfcxn4vlmj89mh0000gn/T/ipykernel_59652/2357794087.py:1: UserWarning: 
********************************************************************************
RAGatouille WARNING: Future Release Notice
--------------------------------------------
RAGatouille version 0.0.10 will be migrating to a PyLate backend 
instead of the current Stanford ColBERT backend.
PyLate is a fully mature, feature-equivalent backend, that greatly facilitates compatibility.
However, please pin version <0.0.10 if you require the Stanford ColBERT backend.
********************************************************************************
  from ragatouille import RAGPretrainedModel


In [5]:

# Run tests for each configuration
for config in CONFIGS:
    # Extract configuration parameters
    chunk_size = config["chunk"]
    reader_name = config["reader"]
    rerank = config["rerank"]
    embeddings = config["embeddings"]

    # Get the reader LLM
    reader_llm = READER_MODELS[reader_name]

    # Create settings name for output file
    settings_name = f"chunk:{chunk_size}_embeddings:{embeddings.replace('/', '~')}_rerank:{rerank}_reader:{reader_name}"
    output_file_name = f"output/rag_{settings_name}.json"

    print(f"\n{'='*60}")
    print(f"Running configuration: {settings_name}")
    print(f"{'='*60}")

    # Load embeddings (will use cache if already exists)
    print("Loading knowledge base embeddings...")
    knowledge_index = load_embeddings(
        RAW_KNOWLEDGE_BASE,
        chunk_size=chunk_size,
        embedding_model_name=embeddings,
    )

    # Setup reranker if needed
    print("Setting up reranker..." if rerank else "Skipping reranker...")
    reranker = None
    if rerank:
        reranker = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

    # Run RAG tests
    print("Running RAG tests...")
    run_rag_tests(
        eval_dataset=eval_dataset,
        llm=reader_llm,
        knowledge_index=knowledge_index,
        output_file=output_file_name,
        reranker=reranker,
        verbose=False,
        test_settings=settings_name,
    )

    # Evaluate answers
    print("Evaluating answers...")
    evaluate_answers(
        output_file_name,
        eval_chat_model,
        evaluator_name,
        evaluation_prompt_template,
    )

    print(f"✓ Completed: {settings_name}")

print(f"\n{'='*60}")
print("All configurations completed!")
print(f"{'='*60}")


Running configuration: chunk:200_embeddings:text-embedding-3-small_rerank:False_reader:gpt-4o-mini
Loading knowledge base embeddings...
Index not found, generating it...
Processing embedding batch 2...
Processing embedding batch 3...
Processing embedding batch 4...
Processing embedding batch 5...
Processing embedding batch 6...
Processing embedding batch 7...
Processing embedding batch 8...
Processing embedding batch 9...
Processing embedding batch 10...
Processing embedding batch 11...
Processing embedding batch 12...
Processing embedding batch 13...
Processing embedding batch 14...
Processing embedding batch 15...
Processing embedding batch 16...
Processing embedding batch 17...
Processing embedding batch 18...
Processing embedding batch 19...
Processing embedding batch 20...
Processing embedding batch 21...
Processing embedding batch 22...
Processing embedding batch 23...
Processing embedding batch 24...
Processing embedding batch 25...
Processing embedding batch 26...
Processing e

100%|██████████| 65/65 [01:49<00:00,  1.68s/it]


Evaluating answers...


100%|██████████| 65/65 [03:09<00:00,  2.92s/it]


✓ Completed: chunk:200_embeddings:text-embedding-3-small_rerank:False_reader:gpt-4o-mini

Running configuration: chunk:200_embeddings:text-embedding-3-small_rerank:True_reader:gpt-4o-mini
Loading knowledge base embeddings...
Setting up reranker...
[Sep 14, 15:02:25] Loading segmented_maxsim_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


...

Running RAG tests...


  0%|          | 0/65 [00:00<?, ?it/s]/Users/manolisfrg/Documents/MyCode/dunderscore_ai/advanced-rag/graphrag_demo/.venv/lib/python3.12/site-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()
/Users/manolisfrg/Documents/MyCode/dunderscore_ai/advanced-rag/graphrag_demo/.venv/lib/python3.12/site-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
  2%|▏         | 1/65 [00:03<03:51,  3.62s/it]/Users/manolisfrg/Documents/MyCode/dunderscore_ai/advanced-rag/graphrag_demo/.venv/lib/python3.12/site-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContext

Evaluating answers...


100%|██████████| 65/65 [02:57<00:00,  2.73s/it]

✓ Completed: chunk:200_embeddings:text-embedding-3-small_rerank:True_reader:gpt-4o-mini

All configurations completed!


In [6]:
import glob
import pandas as pd
import json
import plotly.express as px
import regex as re



# Load all results
outputs = []
for file in glob.glob("output/*.json"):
    output = pd.DataFrame(json.load(open(file, "r")))
    output["settings"] = file
    outputs.append(output)

result = pd.concat(outputs)

# Process scores
result["eval_score_GPT4"] = result["eval_score_GPT4"].apply(
    lambda x: int(x) if isinstance(x, str) else 1
)
result["eval_score_GPT4"] = (result["eval_score_GPT4"] - 1) / 4

# Calculate average scores
average_scores = result.groupby("settings")["eval_score_GPT4"].mean()

# Filter to only include configs we actually ran
config_files = []
for config in CONFIGS:
    settings_name = f"chunk:{config['chunk']}_embeddings:{config['embeddings'].replace('/', '~')}_rerank:{config['rerank']}_reader:{config['reader']}"
    file_pattern = f"output/rag_{settings_name}.json"
    config_files.append(file_pattern)

# Filter average_scores to only include our configs
filtered_scores = average_scores[average_scores.index.isin(config_files)]

# Convert to percentage
scores_percentage = filtered_scores * 100


# Create readable labels
def create_label(filepath):
    """Extract readable label from filepath"""
    # Extract components from filename
    import re

    pattern = r"chunk:(\d+)_embeddings:([^_]+)_rerank:(\w+)_reader:([^.]+)"
    match = re.search(pattern, filepath)

    if match:
        chunk, embeddings, rerank, reader = match.groups()
        # Create concise label
        rerank_str = "w/ rerank" if rerank == "True" else ""
        return f"Chunk {chunk}, {reader}{', ' + rerank_str if rerank_str else ''}"
    return filepath


# Create DataFrame for plotting
df_plot = pd.DataFrame(
    {
        "Configuration": [create_label(idx) for idx in scores_percentage.index],
        "Accuracy": scores_percentage.values,
        "Chunk Size": [
            int(re.search(r"chunk:(\d+)", idx).group(1))
            for idx in scores_percentage.index
        ],
        "Reader Model": [
            re.search(r"reader:([^.]+)", idx).group(1)
            for idx in scores_percentage.index
        ],
    }
)

# Sort by accuracy for better visualization
df_plot = df_plot.sort_values("Accuracy", ascending=True)

# Create grouped bar chart
fig = px.bar(
    df_plot,
    x="Accuracy",
    y="Configuration",
    orientation="h",
    color="Reader Model",
    text="Accuracy",
    title="<b>RAG Performance: Chunk Size and Reader Model Comparison</b>",
    labels={"Accuracy": "Accuracy (%)"},
    color_discrete_map={"gpt-4o-mini": "#3498db", "gpt-4o": "#e74c3c"},
)

fig.update_layout(
    width=900,
    height=400 + len(df_plot) * 40,  # Dynamic height based on number of configs
    xaxis_range=[0, 100],
    xaxis=dict(ticksuffix="%"),
    font=dict(size=12),
    showlegend=True,
    legend=dict(
        title="Reader Model",
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="left",
        x=1.02,
    ),
)

fig.update_traces(texttemplate="%{x:.1f}%", textposition="outside")

fig.show()

# Print summary table
print("\n" + "=" * 60)
print("EVALUATION SUMMARY")
print("=" * 60)

summary_df = df_plot[["Configuration", "Accuracy"]].copy()
summary_df["Accuracy"] = summary_df["Accuracy"].apply(lambda x: f"{x:.1f}%")
summary_df = summary_df.sort_values("Configuration")

print(summary_df.to_string(index=False))

# Find best configuration
best_config = df_plot.loc[df_plot["Accuracy"].idxmax()]
print("\n" + "-" * 60)
print(f"✨ Best Configuration: {best_config['Configuration']}")
print(f"   Accuracy: {best_config['Accuracy']:.1f}%")
print("-" * 60)


EVALUATION SUMMARY
                    Configuration Accuracy
           Chunk 200, gpt-4o-mini    73.5%
Chunk 200, gpt-4o-mini, w/ rerank    73.1%

------------------------------------------------------------
✨ Best Configuration: Chunk 200, gpt-4o-mini
   Accuracy: 73.5%
------------------------------------------------------------
